In [1]:
%reload_ext autoreload
%autoreload 2

from src.acnets.deep import LEMONDataModule, Julia2018DataModule
import numpy as np
import torch
from torch import nn
import torch.nn.functional  as F
import pywt
import matplotlib.pyplot as plt
import seaborn as sns

from src.acnets.deep.cvae import CVAE

In [2]:
datamodule = LEMONDataModule(n_subjects=8, test_ratio=None)
# datamodule = Julia2018DataModule()
datamodule.setup()
batch = datamodule.train[:]

In [3]:
datamodule = LEMONDataModule(n_subjects=8, test_ratio=None)
datamodule.setup()

batch = datamodule.train[:]
x_wavelets = batch[5] # -> shape: (subjects, wavelets, regions)

n_channels = x_wavelets.shape[2]
n_wavelets = 32
n_embeddings = 16

x = x_wavelets[:, :n_wavelets, :].transpose(1, 2)  # -> shape: (subjects, regions, wavelets)

h, x_recon, loss = CVAE(n_channels, n_embeddings)(x)
x.shape, h.shape, x_recon.shape, loss


(torch.Size([8, 160, 32]),
 torch.Size([8, 16]),
 torch.Size([8, 160, 32]),
 tensor(3.2736, grad_fn=<AddBackward0>))

In [13]:
%reload_ext autoreload
%autoreload 2

from src.acnets.deep import MultiHeadWaveletModel, LEMONDataModule, Julia2018DataModule

lemon_datamodule = LEMONDataModule(
    atlas='dosenbach2010', kind='partial correlation',
    test_ratio=.1, val_ratio=.05,
    n_subjects=201, batch_size=201)
lemon_datamodule.setup()

julia2018_datamodule = Julia2018DataModule(
    atlas='dosenbach2010', kind='partial correlation',
    test_ratio=.5, batch_size=16)

n_regions = lemon_datamodule.train[0][0].shape[1]
n_wavelets = 96
n_embeddings = 128
segment_length = 32

model = MultiHeadWaveletModel(n_regions, n_wavelets, n_embeddings, segment_length)

trainer = model.fit(lemon_datamodule, max_epochs=10000, with_classifier=False)
tuner = model.fit(julia2018_datamodule, max_epochs=1000, with_classifier=True)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/morteza/miniforge3/envs/acnets/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:186: .fit(ckpt_path="last") is set, but there is no last checkpoint available. No checkpoint will be loaded.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name              ┃ Type               ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ accuracy          │ MulticlassAccuracy │      0 │
│ 1 │ feature_extractor │ CVAE               │ 11.1 M │
│ 2 │ cls_head          │ Classifier         │ 24.9 K │
└───┴───────────────────┴────────────────────┴────────┘

Trainable params: 11.1 M                                                                                           
Non-trainable params: 24.9 K                                                                                       
Total params: 11.1 M                                                                                               
Total estimated model params size (MB): 44

Output()

RuntimeError: Calculated padded input size per channel: (1). Kernel size: (5). Kernel size can't be greater than actual input size